In [ ]:
# !git clone https://github.com/dsa-playground/tvt2023.git
# %cd /content/tvt2023/
# !git pull
# !pip install -r requirements.txt -t "tvt2023"
# !pip install pyaml-env

# Inleiding

### Use case 'Titanic'
Ruim honderd jaar geleden (1912) zonk de Titanic, vier uur nadat het schip op een ijsberg was gelopen. Slechts een derde deel van de opvarende overleefde deze ramp. Veel van deze gebeurtenis is vastgelegd, waaronder ook een dataset van passagiers. Deze dataset leent zich goed voor een introductie in de Data Science. Kan een algoritme voorspellen of een passagier overleeft? En, als je jezelf toevoegd, zou jij het dan overleefd hebben?

![Laatste foto van de Titanic](https://raw.githubusercontent.com/dsa-playground/tvt2023/main/images/lastphoto_titanic.png)

### Instructies omgeving
Voor deze workshop werken we in Google Colab. Dit is een online ontwikkelomgeving waarin je eenvoudig kunt experimenteren. Het notebook wat we voorbereid hebben staat al klaar. In een notebook staan cellen met ofwel code, tekst of afbeeldingen. Om de code in een cel te draaien zijn er twee mogelijkheden:
- Play-knop links van de cel
- Ctrl + Enter

### 0. Klaarzetten software
Data Science heeft een sterke component met Computer Science. De meeste programmatuur zit op de achtergrond, maar om gebruik te maken van de functionaliteiten worden de functies en instellingen geladen in de cel hieronder. 

In [ ]:
# Imports
import warnings
import pandas as pd
from scripts.preprocess.preprocess_frontend import laden_data, opschonen_data, numeriek_maken_data, voeg_passagiers_toe
from scripts.EDA.eda import basis_feiten, visualisatie_ticketklasse, visualisatie_opstapplaats, visualisatie_leeftijd ,visualisatie_leeftijd_geslacht ,visualisatie_familieleden
from scripts.modeling.modeling_frontend import experimenteer_met_aantal_buren, verdieping_specifiek_model, voorspelling_genereren
from scripts.evaluation.evaluation import geef_belangrijkste_variabelen 

# Settings
# settings for pandas
pd.set_option("display.max.columns",None) # alle kolommen tonen
pd.set_option("display.max.rows",500)    # eerste 500 rijen tonen
pd.set_option("display.precision", 2)     # precisie van de kolommen aanpassen
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) # floats output tot 3 decimalen
pd.set_option('display.max_colwidth', None)

# Disable warnings
warnings.filterwarnings('ignore')

### 1. Data Science Proces
Data Science is geen doel op zich. Het doel is antwoord vinden voor een vraagstuk. Om te borgen dat Data (Science) producten aansluiten bij de wensen van een klant kan men het CRISP-DM proces (Cross-Industry Standard Process for Data Mining) volgen. 

<img src=https://raw.githubusercontent.com/dsa-playground/tvt2023/main/images/CRISP-DM.png width=400 height=400 alt="CRISP-DM">

Dit proces doorloopt de volgende stappen:
- Business understanding: Vinden van de hypothese en context.
- Data understanding: Verzamelen van relevante data.
- Data preparation: Aanpassen data zodat deze bruikbaar is voor algoritme.
- Modeling: Opzetten/inrichten algoritme om antwoord te vinden op hypothese. 
- Evaluation: Reflecteren of resultaat model hypothese verwerpt of aanneemt.
- Deployment: Naar productieomgeving brengen (zorg dragen dat model meermaals gebruikt kan worden). 

### 2. Business Understanding
Het vraagstuk nu concentreert zich op wel/niet overleven van de Titanic. Oftewel:
- *Kan een algoritme voorspellen of een passagier de Titanic overleefd?*

En... door onszelf toe te voegen kijken of **wij** dit hadden overleefd!

### 3. Data Understanding
Er is een dataset beschikbaar met informatie van passagiers. Deze is gesplitst in twee datasets:

1. Train: Dataset met passagiers én informatie of zij wel/niet overleefd hebben
2. Test: Dataset met passagiers *zonder* informatie of zij wel/niet overleefd hebben

Laten we eens kijken naar de train dataset. Draai de code door op de playknop te drukken. 

In [ ]:
df_train, df_test = laden_data()
display(df_train.head())

Zoals je ziet is de dataset in het Engels en soms wat cryptisch weergegeven.   
Om het iets eenvoudiger te maken, transformeren we de dataset naar iets begrijpelijkere taal en zetten we vergelijkbare informatie bij elkaar.

In [ ]:
df_train_clean, df_test_clean = opschonen_data(df_train, df_test)
display(df_train_clean.head())

De data die we nu zien heeft informatie over:
* Passagier: ID, naam, geslacht, leeftijd, opstapplaats, aantal kinderen, aantal overige familieleden, totaal aantal familieleden
* Reisinformatie: Opstapplaats, ticket nummer, ticket klasse, cabine nummer
* Overleefd ja/nee

Wat we willen voorspellen is of mensen het overleefd hebben. De kolom 'Overleefd' is wat we noemen 'target-variabele'. De andere variabelen zijn mogelijk de verklarende variabelen. Om te kijken of er waarde zit in de variabelen, doen we een verkennende gegevensanalyse (EDA: exploratory data analysis).

In [ ]:
basis_feiten(df=df_train_clean)

In [ ]:
visualisatie_ticketklasse(df=df_train_clean)

In [ ]:
visualisatie_opstapplaats(df=df_train_clean)

In [ ]:
visualisatie_leeftijd(df=df_train_clean)

In [ ]:
visualisatie_leeftijd_geslacht(df=df_train_clean)

In [ ]:
visualisatie_familieleden(df=df_train_clean)

### 4. Data preparation
Uit de verkennende gegevensanalyse blijkt dat we de dataset moeten aanpassen voordat we deze kunnen gebruiken in een voorspelling. De volgende aanpassingen zijn nodig:
- Verwijderen kolommen met geen relevante data (zoals 'ticket_nummer')
- Verwijderen kolommen met veel missende data (zoals 'cabine_nummer')
- Vullen van missende waarden waar mogelijk (zoals bij 'leeftijd')
- Afronden van leeftijd (34,5 jaar = 34 jaar)
- Numeriek maken van waarden (geslacht, opstapplaats, ticket_klasse, overleefd)
- Nieuwe index maken (unieke combinatie per rij)

Dit leidt tot de volgende datasets (train & test):

In [ ]:
df_train_num, df_test_num = numeriek_maken_data(df_train_clean, df_test_clean)
display(df_train_num.head(), df_test_num.head())

De dataset is bijna klaar voor het toepassen van een model, maar... we willen natuurlijk ook weten of je het zelf overleefd zou hebben! Laten we onszelf toevoegen aan de dataset.

In [ ]:
df_train_extended, df_test_extended = voeg_passagiers_toe(df_train_num, df_test_num)
display(df_test_extended.tail())

### 5. Modeling
Zoals in de presentatie besproken zijn er verschillende algoritmes om vraagstukken op te lossen.  
 In deze workshop gaan we voor een classificatie-algoritme: KNN (K Nearest Neighbor). 
 
 Hoe dit algoritme werkt laat zich het beste uitleggen op basis van een animatie:

![knn](https://raw.githubusercontent.com/dsa-playground/tvt2023/main/images/knn.gif)

Wat het algoritme doet is het bepaald tot welke groep een 'onbekend' punt behoort, door de dichtsbijzijnde punten te zoeken.  
De meerderheid van de dichtstbijzijnde punten, bepalen wat het 'onbekende' punt wordt. 

Een variabele waar je in het trainen van een model mee experimenteert is bijvoorbeeld het aantal buren.  
Deze 'buren' zijn het aantal dichtstbijzijnde punten waarmee rekening wordt gehouden.

Natuurlijk is dit een vereenvoudigd voorbeeld met slechts 2 variabelen (X1 en X2).  
In het geval van de Titanic dataset zijn er veel meer variabelen en dus dimensies. 

Wij als mensen kunnen tot 3 dimensies vrij aardig visualiseren, daarna is eigenlijk al niet meer te doen.  
Gelukkig wordt een algoritme niet geremd door meer dimensies. 

Om ons eigen model te trainen, moeten we ook ingeven met hoeveel buren er rekening gehouden moet worden.
We weten echter niet bij hoeveel buren het beste model gevonden wordt. 

Gelukkig is dit allemaal uit te rekenen, dus laten we experimenteren met een aantal verschillende buren (van 1 tot 15):

In [ ]:
df_experiment = experimenteer_met_aantal_buren(df=df_train_extended, ondergrens=1, bovengrens=15)
display(df_experiment)

### 6. Evaluation

Maar hoe weet je nu wanneer een voorspelling 'goed genoeg' is?  
En hoe vergelijk je verschillende voorspellingen met elkaar?

Voor de evaluatie van voorspellingen zijn gelukkig verschillende metrieken ontwikkeld.  
In deze casus is gekozen voor de metriek genaamd 'nauwkeurigheidsscore', deze geeft aan hoeveel procent juist voorspeld is. 

De basis voor deze nauwkeurigheidsscore is in de cel hieronder gevisualiseerd, door middel van een zogenaamde 'verwarringsmatrix'.

In [ ]:
df_confusion_matrix = verdieping_specifiek_model(df=df_train_extended, aantal_buren=3)
display(df_confusion_matrix)

In bovenstaande verwarringsmatrix worden twee assen met elkaar gecombineerd:

* Wel/niet overleefd
* Werkelijke waarde vs voorspelde waarde

Wanneer je deze assen tegen elkaar afzet levert dat vier verschillende categorieën op:

* 87 passagiers die zijn overleden en waarbij de voorspelling ook overleden is (juiste voorspelling, 'True Negatives')
* 18 passagiers die zijn overleden maar waarbij de voorspelling overleefd is (onjuiste voorspelling, 'False Positives')
* 28 passagiers die het hebben overleefd maar waarbij de voorspelling overleden is (onjuiste voorspelling, 'False Negatives')
* 45 passagiers die het hebben overleefd en waarbij de voorspelling ook overleefd is (juiste voorspelling, 'True Positives')

De nauwkeurigheidsscore wordt berekend door de aantallen van de juiste voorspellingen te delen door het totaal. In het voorbeeld van 3 buren:

${(87+45) \over (87+18+28+45)} = {132 \over 178} = 0.742$

Het is in de evaluatiefase belangrijk om goed na te denken over de metriek die je kiest. De nauwkeurigsheiddscore houd bijvoorbeeld rekening met zowel de 'True Negatives' als de 'True Positives'. Maar er zijn, zeker in de zorg, genoeg voorbeelden te vinden waarbij je juist de nadruk wilt leggen op maar één van die twee. En dat heeft vervolgens ook weer invloed op welk algoritme het beste het uit de bus komt.

Nu terug naar onze use case. Laten we de passagiers voorspellen waarvoor het onbekend is, waaronder ons zelf! 

In [ ]:
df_voorspelling = voorspelling_genereren(X=df_test_extended)
display(df_voorspelling.tail())

Voor sommige is het resultaat wellicht teleurstellend.   
Maar wees gerust, er zijn een aantal variabelen doorslaggevend en bij een goed doorlopen CRISP-DM proces heb je deze in beeld en kun (vroegtijdig) ingrijpen.  

In onderstaande visualisatie kun je zien welke variabelen doorgeslaggevend waren.

In [ ]:
geef_belangrijkste_variabelen(df_train_num)

### 7. Deployment

Het CRISP-DM proces is een iteratief proces. 

In elke stap kan een inzicht zijn wat leidt tot nieuwe afstemming of herzien van instellingen.  
 Met als doel dat het resultaat voldoet aan de eisen/verwachtingen bij de klant. 

Wanneer alles naar wens is dan kan men overgaan tot de laatste fase: 'Deployment'.   
Deze fase is niet te onderschatten, want hierin komen weer hele andere vaardigheden kijken dan puur het maken van voorspelling. 

Een gemaakte voorspelling moet bijvoorbeeld regelmatig opnieuw geëvalueerd worden op basis van nieuwe data.   
Gelukkig hoeven we daar met de Titanic niet mee aan de slag, want dit was puur een introductie tot ons mooie vakgebied!